In [ ]:
import json

with open("./data-raw-dump.json", "r") as f:
    data = json.load(f)

recipes = data["recipe"]
items = data["item"]
print(len(recipes))
print(len(items))
# for recipe in recipes:
#     print(recipe)

In [ ]:
# def sort_recipes(recipes):
#     def sort_key(recipe):
#         # 1. 按组(group)排序
#         group = recipe.get('group', '')
        
#         # 2. 按子组(subgroup)排序
#         subgroup = recipe.get('subgroup', '')
        
#         # 3. 按order字符串排序
#         order = recipe.get('order', '')
        
#         # 解析order字符串
#         order_parts = order.split('-')
#         main_order = order_parts[0] if order_parts else ''
#         sub_order = '-'.join(order_parts[1:]) if len(order_parts) > 1 else ''
        
#         # 4. 如果所有其他条件相同,按名称排序
#         name = recipe.get('name', '')
        
#         return (group, subgroup, main_order, sub_order, name)

#     return sorted(recipes, key=sort_key)

with open("recipes copy.json", "r", encoding="utf-8") as f:
    data = json.load(f)

recipes = data["recipes"]
prototypes = []
for recipe in recipes:
    if recipes[recipe]["group"] == "logistics":
        if recipes[recipe]["name"].startswith("request-"):
            # 删除request-打头的配方
            del recipes[recipe]

with open("recipes copy.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)
            

In [17]:
import json

with open("recipes copy.json", "r", encoding="utf-8") as f:
    data = json.load(f)

recipes = data["recipes"]
recipes_to_keep = {}

for recipe_key, recipe_value in recipes.items():
    if not recipe_value["name"].startswith("request-"):
        recipes_to_keep[recipe_key] = recipe_value

data["recipes"] = recipes_to_keep

with open("recipes copy1.json", "w", encoding="utf-8") as f:
    json.dump(data, f,ensure_ascii=False, indent=2)

In [3]:
import json
import sqlite3
from pathlib import Path

# 读取JSON文件
json_file = Path('factorio_data.json')
with json_file.open('r', encoding='utf-8') as f:
    data = json.load(f)

# 连接到SQLite数据库（如果不存在则创建）
conn = sqlite3.connect('factorio.db')
cursor = conn.cursor()

# 创建表（注意：将 'order' 改为 'order_value'）
cursor.execute('''
CREATE TABLE IF NOT EXISTS groups (
    id TEXT PRIMARY KEY,
    name TEXT,
    translated_name TEXT,
    order_value TEXT,
    type TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS subgroups (
    id TEXT PRIMARY KEY,
    name TEXT,
    translated_name TEXT,
    order_value TEXT,
    group_id TEXT,
    FOREIGN KEY (group_id) REFERENCES groups (id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS items (
    id TEXT PRIMARY KEY,
    name TEXT,
    translated_name TEXT,
    type TEXT,
    order_value TEXT,
    subgroup_id TEXT,
    FOREIGN KEY (subgroup_id) REFERENCES subgroups (id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS recipes (
    id TEXT PRIMARY KEY,
    name TEXT,
    translated_name TEXT,
    category TEXT,
    energy REAL,
    order_value TEXT,
    enabled INTEGER,
    subgroup_id TEXT,
    FOREIGN KEY (subgroup_id) REFERENCES subgroups (id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS recipe_ingredients (
    recipe_id TEXT,
    item_id TEXT,
    amount REAL,
    FOREIGN KEY (recipe_id) REFERENCES recipes (id),
    FOREIGN KEY (item_id) REFERENCES items (id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS recipe_products (
    recipe_id TEXT,
    item_id TEXT,
    amount REAL,
    probability REAL,
    is_main_product INTEGER,
    FOREIGN KEY (recipe_id) REFERENCES recipes (id),
    FOREIGN KEY (item_id) REFERENCES items (id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS producers (
    id TEXT PRIMARY KEY,
    name TEXT,
    translated_name TEXT,
    crafting_speed REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS producer_categories (
    producer_id TEXT,
    category TEXT,
    FOREIGN KEY (producer_id) REFERENCES producers (id)
)
''')

# 插入数据
for table_name in ['groups', 'subgroups', 'items', 'recipes', 'recipe_ingredients', 'recipe_products', 'producers', 'producer_categories']:
    if table_name in data:
        for item in data[table_name]:
            # 将 'order' 键重命名为 'order_value'
            if 'order' in item:
                item['order_value'] = item.pop('order')
            
            placeholders = ', '.join(['?'] * len(item))
            columns = ', '.join(f'"{k}"' for k in item.keys())  # 将列名用双引号括起来
            sql = f'INSERT OR REPLACE INTO {table_name} ({columns}) VALUES ({placeholders})'
            cursor.execute(sql, list(item.values()))

# 提交更改并关闭连接
conn.commit()
conn.close()

print("Data has been successfully imported into the SQLite database.")

Data has been successfully imported into the SQLite database.
